# `twitter_blocklist`

Manage list of blocked users on Twitter.

## Install the package

This should only be necessary the first time your run the notebook and once in a while to update it

In [ ]:
# NBVAL_IGNORE_OUTPUT
!pip install --upgrade twitter_blocklist

Ignore the `# NBVAL_IGNORE_OUTPUT` lines, they are just needed for automated testing.

## Setup authentication

This software uses the Twitter API to connect to Twitter on your behalf to read and optionally modify the list of blocked users. Therefore it needs to authenticate to Twitter, this is achieved creating an "App" and copying special authentication token strings.

First follow the instructions by the [`python-twitter` project](https://python-twitter.readthedocs.io/en/latest/getting_started.html) on how to create an "App", in this phase Twitter might ask you to create a Developer account, in the case, don't worry it is pretty quick.

Under "App permissions", you need to set "Read and Write", otherwise you would get the error "Read-only app" cannot POST when trying to import a blocklist.

Once it is done, copy-paste the "API Key and Secret" as consumer key and secret, then click on "Generate" under "Access token & Secret" and paste them below:

In [2]:
# NBVAL_IGNORE_OUTPUT
%rm twitter_keys.toml

In [3]:
%%file twitter_keys.toml

consumer_key='xxxxxxxxxxxxxxxxxxxx'
consumer_secret='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token_key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token_secret='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

Writing twitter_keys.toml


Alternatively, you can define the equivalent environment variables (they have precedence over the toml file):

    export TWITTER_CONSUMER_KEY=''
    export TWITTER_CONSUMER_SECRET=''
    export TWITTER_ACCESS_TOKEN_KEY=''
    export TWITTER_ACCESS_TOKEN_SECRET=''

We can now test if the authentication is working by getting a list of tweets by the authenticated users and their follows:

In [4]:
# NBVAL_IGNORE_OUTPUT
from twitter_blocklist import authenticate
api = authenticate()
api.GetHomeTimeline()

[Status(ID=1315688655298523136, ScreenName=blocklistbot, Created=Mon Oct 12 16:19:54 +0000 2020, Text='first tweeting test')]

## Notice about rate limits

Consider that Twitter rate-limits their APIs, I have setup the client to automatically sleep in case of a rate-limiting error, in case that happens, just leave the script running and it will complete at some point. For example exporting the blocks needs to make 1 request every 5000 blocked IDs, so you could hit the limit of 15 requests every 15 minutes, in that case the script will sleep for 15 minutes and then resume.

## Export the current list of blocks to a `csv` file

The standard format is one block per line, first the integer Twitter profile ID and then the screen name:

In [5]:
# NBVAL_IGNORE_OUTPUT
!twitter_blocklist --export my_blocks.csv

In [6]:
with open("my_blocks.csv", "r") as f:
    print(f.readlines())

['6253282,"TwitterAPI"\n', '436266454,"TwitterMovies"\n', '783214,"Twitter"\n']


## Unblock users from a `csv` file

In case we need to undo a list of blocks done with `twitter_blocklist`, we can perform the opposite operation:

In [7]:
# NBVAL_IGNORE_OUTPUT
!twitter_blocklist --unblock my_blocks.csv

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [8]:
api.GetBlocksIDs()

[]

## Import blocks from a `csv` file

We can share your blocks with other users, or collaborate together on maintaining a list, or import a list previously exported from BlockTogether:

In [9]:
# NBVAL_IGNORE_OUTPUT
!twitter_blocklist my_blocks.csv

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [10]:
api.GetBlocksIDs()

[6253282, 436266454, 783214]

## Share a list of blocks with Github or Pastebin

The easiest way to publish a list of blocks is create an account on <https://pastebin.com> and paste the content of the `csv` file there, see for example <https://pastebin.com/NndFiwU3> and then share the "RAW" url (right click on the "RAW" button and copy the link address), in this case <https://pastebin.com/raw/NndFiwU3>.

Same with Gist <https://gist.github.com> or creating a dedicated Github repository where multiple people can contribute, see for example <https://github.com/zonca/example_blocklist>.
Also on Github, we need to share the "RAW" url for `twitter_blocklist`, in the example repository above, it would be:

* <https://raw.githubusercontent.com/zonca/example_blocklist/main/a_blocklist.csv>

We could download those files, edit them locally and then import them using the command above, but `twitter_blocklist` also support downloading them on the fly, so that we can always grab the most updated version:

In [11]:
len(api.GetBlocksIDs())

3

In [12]:
blocklists_urls = [
    "https://pastebin.com/raw/NndFiwU3",
    "https://raw.githubusercontent.com/zonca/example_blocklist/main/a_blocklist.csv"
]

In [13]:
# NBVAL_IGNORE_OUTPUT
for url in blocklists_urls:
    !twitter_blocklist $url

100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


The same can be executed in the terminal as:

    > twitter_blocklist https://pastebin.com/raw/NndFiwU3
    > twitter_blocklist https://raw.githubusercontent.com/zonca/example_blocklist/main/a_blocklist.csv

In [14]:
len(api.GetBlocksIDs())

7

As before, we can undo the operation adding the `--unblock` option:

In [15]:
# NBVAL_IGNORE_OUTPUT
for url in blocklists_urls:
    !twitter_blocklist --unblock $url

100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [16]:
len(api.GetBlocksIDs())

3

## Handling lists

**Notice about lists**: if you create a list of users and then block them, they are removed from the list. So this technique is only useful if you want to block a list maintained by someone else.

Another possibility for sharing blocks is to use Twitter lists, if you create a list using the Twitter web interface or any app, see this example list:

https://twitter.com/i/lists/1315715236456853504

You notice that the URL has a long string of numbers, that is the List ID and we can use that to block/unblock all the accounts in the list:

In [17]:
len(api.GetBlocksIDs())

3

In [18]:
# NBVAL_IGNORE_OUTPUT
!twitter_blocklist --list 1315715236456853504

100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [19]:
len(api.GetBlocksIDs())

5

In [20]:
# NBVAL_IGNORE_OUTPUT
!twitter_blocklist --unblock --list 1315715236456853504

100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [21]:
len(api.GetBlocksIDs())

3